In [1]:
import numpy as np
import pygsti as pig
import matplotlib.pyplot as plt
import random
import time
import pandas
from scipy.linalg import expm, sqrtm
from tqdm import tqdm
# from qutip import *

from pygsti.modelmembers.operations import LindbladErrorgen as _LinbladErrorgen
from pygsti.modelmembers.operations import EmbeddedErrorgen as _EmbeddedErrorgen
from pygsti.modelmembers.operations import ExpErrorgenOp as _ExpErrorgenOp
from pygsti.modelmembers.operations import ComposedOp as _ComposedOp
import  pygsti.modelmembers as _mdmb
import pygsti.processors as _proc

from pygsti.tools import pdftools as _pdftools

from pygsti.circuits.gstcircuits import create_lsgst_circuits

from pygsti.models.memberdict import OrderedMemberDict as _OrderedMemberDict
from pygsti.baseobjs.label import Label as _Label, CircuitLabel as _CircuitLabel
from pygsti.circuits import Circuit
import pygsti.circuits as _circ
import pygsti.models as _mdl
from pygsti.modelmembers import operations as _op

In [2]:
def random_circuit(depth, proc_spec):
    """
    works for any 1-qubit model pack and the XYZICnot 2-qubit pack
    """
    layers = []
    for i in range(depth):
        gate = random.choice(proc_spec.gate_names)
        if gate == '(idle)':
            pass
        elif gate == 'Gcnot':
            layers.append((gate, 0, 1))
        else:
            layers.append((gate, random.choice(proc_spec.qubit_labels)))
    return Circuit(layers, proc_spec.qubit_labels)

In [3]:
def vector_from_outcomes(outcomes, num_outcomes):
    vecout = np.zeros((num_outcomes))
    for key in outcomes.keys():
        vecout[int(key[0], 2)] = outcomes[key]
    return(vecout)

def matrix_from_jacob(jacob, num_outcomes):
    matout = np.zeros((num_outcomes, len(jacob['0'*int(np.log2(num_outcomes))])))
    for key in jacob.keys():
        matout[int(key[0], 2), :] = np.array(jacob[key])
    return matout

def tensor_from_hessian(hessian, hilbert_dims):
    """
    returns a 3d-array that when dotted into the state returns the jacobian 
    """
    num_params = len(hessian['0'*int(np.log2(hilbert_dims))])
    tensor_out = np.zeros((hilbert_dims, num_params, num_params))
    for key in hessian.keys():
        tensor_out[int(key[0], 2), :, :] = hessian[key]
    return tensor_out

In [4]:
def compare_models(true_model, model1, model2, circuit_list, name1='x', name2='y'):
    distribution1 = []
    distribution2 = []
    mx = 0
    for idx, circ in enumerate(circuit_list): 
        hilbert_dims = 2**circ.width
        true_outcomes = true_model.probabilities(circ)
        outcome_set1 = model1.probabilities(circ)
        outcome_set2 = model2.probabilities(circ)
        dist1 = _pdftools.tvd(true_outcomes, outcome_set1)
        dist2 = _pdftools.tvd(true_outcomes, outcome_set2)
        if dist1 > mx:
            mx = dist1
        if dist2 > mx: 
            mx = dist2
        vtrue = vector_from_outcomes(true_outcomes, hilbert_dims)
        v1 = vector_from_outcomes(outcome_set1, hilbert_dims)
        v2 = vector_from_outcomes(outcome_set2, hilbert_dims)
        distribution1.append(vtrue - v1)
        distribution2.append(vtrue - v2)
        plt.scatter(dist1, dist2, s=75, c='black')
    plt.xlabel(name1)
    plt.ylabel(name2)
    plt.plot((0, mx), (0, mx), c='black')
    plt.show()
    return (distribution1, distribution2)

In [5]:
def calculate_jacobs(circ_list, model):
    jacobs = {}
    for circ in circ_list:
        hilbert_dims = 2**circ.width
        jacobs[circ] = matrix_from_jacob(model.sim.dprobs(circ), hilbert_dims)
    return jacobs

In [6]:
from pygsti.processors import CliffordCompilationRules as CCR

def make_rb_param(noise_model, model_pack, length_powers=5, circuits_at_length=10):
    pspec = model_pack.processor_spec()
    
    depths = [2**i for i in range(length_powers)]
    
    compilations = {'absolute': CCR.create_standard(pspec, 'absolute', ('paulis', '1Qcliffords'), verbosity=0),            
                'paulieq': CCR.create_standard(pspec, 'paulieq', ('1Qcliffords', 'allcnots'), verbosity=0)}

    design = pig.protocols.DirectRBDesign(model_pack.processor_spec(), compilations, depths, circuits_at_length, qubit_labels=model_pack.processor_spec().qubit_labels, sampler='edgegrab', 
                                           samplerargs=[0.5], randomizeout=True,
                                           citerations=20)
    pig.io.write_empty_protocol_data(design, 'RB_Data', clobber_ok=True)
    pig.io.fill_in_empty_dataset_with_fake_data(noise_model, 'RB_Data/data/dataset.txt', num_samples=1000)
    data = pig.io.load_data_from_dir('RB_Data')
    protocol = pig.protocols.RB()
    results = protocol.run(data)
    return (results.fits['full'].estimates['r'], data)

In [7]:
def make_dirichlet_covar(counts, hilbert_dims):
    total_counts = sum([counts[key] for key in counts.keys()])    
    count_vec = np.ones(hilbert_dims)
    for key in counts.keys():
        count_vec[int(key[0], 2)] += counts[key]
    prefactor = 1/( (total_counts + hilbert_dims)**2 * (total_counts + hilbert_dims + 1) )
    meas_covar = prefactor*(
        (total_counts + hilbert_dims)*np.diag(count_vec) - np.outer(count_vec, count_vec)
    )
    return meas_covar

In [8]:
def make_multinom_covar(prob_vec):
    return np.eye(len(prob_vec)) - np.outer(prob_vec, prob_vec)

In [9]:
from scipy.linalg import block_diag

class ExtendedKalmanFilter():
    def __init__(self, model, P0):
        self.model = model
        self.P = P0
        
    def update(self, circ_list, data_set, jdict, hdict, stab_noise=None, max_itr=1, itr_eps=1e-4):
        
        
        prior_covar = self.P
        prior_state = self.model.to_vector()
        
        for itr in range(max_itr):
            Smat = np.zeros((0,0))
            total_innov = np.zeros((0))
            total_jac = np.zeros((0,len(self.model.to_vector())))
        
            for circ in circ_list:
                counts = data_set[circ].counts
                total_counts = sum([counts[key] for key in counts.keys()])    
                hilbert_dims = 2**(circ.width)
                prior_state = self.model.to_vector()

                # 0) find the mean estimate for the circuit outcome under the self.model
                p_model = np.ones(hilbert_dims)
                probs = self.model.probabilities(circ)
                for key in probs.keys():
                    p_model[int(key[0], 2)] += total_counts*probs[key]
                p_model = (1/(total_counts+hilbert_dims))*p_model

                # calculate jacobian
                jacob = jdict[circ] + hdict[circ]@prior_state

                # 1) calculate your observation frequencies
                observation = np.ones(hilbert_dims)
                for key in counts.keys():
                    observation[int(key[0], 2)] += counts[key]
                observation = (1/(total_counts+hilbert_dims))*observation

                # 2) calculate the covaraiance of the observation and add model noise
                meas_covar = (1/(total_counts+hilbert_dims))*make_multinom_covar(observation)
                if stab_noise is not None:
                    meas_covar += stab_noise
                    
                # 3) Kalman gain
                smat = np.linalg.pinv(jacob@prior_covar@jacob.T + meas_covar, 1e-6)
                
                innovation = observation - p_model

                Smat = block_diag(Smat, smat)
                total_innov = np.hstack([total_innov, innovation])
                total_jac = np.vstack([total_jac, jacob])
            Kgain = prior_covar@total_jac.T@Smat
            post_state = prior_state + Kgain@total_innov
            self.P = prior_covar - Kgain@total_jac@prior_covar
            if np.linalg.norm(post_state - prior_state) < itr_eps:
                break
        self.model.from_vector(post_state)

In [10]:
from scipy.linalg import block_diag

class ExtendedKalmanFilter2():
    def __init__(self, model, P0):
        self.model = model
        self.P = P0
        
    def update(self, circ_list, data_set, jdict, hdict, stab_noise=None, max_itr=1, itr_eps=1e-4):
        
        
        prior_covar = self.P
        prior_state = self.model.to_vector()
        
        for itr in range(max_itr):
            Smat = np.zeros((0,0))
            total_innov = np.zeros((0))
            total_jac = np.zeros((0,len(self.model.to_vector())))
        
            for circ in circ_list:
                counts = data_set[circ].counts
                total_counts = sum([counts[key] for key in counts.keys()])    
                hilbert_dims = 2**(circ.width)
                prior_state = self.model.to_vector()

                # 0) find the mean estimate for the circuit outcome under the self.model
                p_model = np.zeros(hilbert_dims)
                probs = self.model.probabilities(circ)
                for key in probs.keys():
                    p_model[int(key[0], 2)] += probs[key]

                # calculate jacobian
                jacob = jdict[circ] + hdict[circ]@prior_state

                # 1) calculate your observation frequencies
                observation = np.ones(hilbert_dims)
                for key in counts.keys():
                    observation[int(key[0], 2)] += counts[key]
                observation = (1/(total_counts+hilbert_dims))*observation

                # 2) calculate the covaraiance of the observation and add model noise
                meas_covar = (1/(total_counts+hilbert_dims))*make_multinom_covar(observation)
                if stab_noise is not None:
                    meas_covar += stab_noise
                    
                # 3) Kalman gain
                smat = np.linalg.pinv(jacob@prior_covar@jacob.T + meas_covar, 1e-6)
                
                innovation = observation - p_model

                Smat = block_diag(Smat, smat)
                total_innov = np.hstack([total_innov, innovation])
                total_jac = np.vstack([total_jac, jacob])
            Kgain = prior_covar@total_jac.T@Smat
            post_state = prior_state + Kgain@total_innov
            self.P = prior_covar - Kgain@total_jac@prior_covar
            if np.linalg.norm(post_state - prior_state) < itr_eps:
                break
        self.model.from_vector(post_state)

In [11]:
from filterpy.kalman import KalmanFilter

class LinearKalmanFilter(KalmanFilter):
    def __init__(self, x0, P0, hilbert_dims):
        super().__init__(len(x0), hilbert_dims)
        self.x = x0
        self.P = P0
        
        self.F = np.eye(len(x0))
        self.Q = np.zeros((len(x0), len(x0)))
        
    def update_filter(self, circ, count_vec, pvec_model, jacob, stab_noise=None):
        
        total_counts = sum(count_vec)
        
        hilbert_dims = 2**circ.width
        
        prediction = (total_counts*pvec_model + np.ones(hilbert_dims))/(total_counts + hilbert_dims)
        observation = (count_vec + np.ones(hilbert_dims))/(total_counts + hilbert_dims)
        
        self.H = jacob
        shot_noise = (1/total_counts)*make_multinom_covar(observation)
        if stab_noise is not None:
            shot_noise += stab_noise
        self.R = shot_noise
        
        self.update(observation - prediction)

In [12]:
def regression_fit(circ_list, data_set, ref_model, jdict):
    pvec = np.zeros((0))
    jmat = np.zeros((0, num_params))
    
    for circ in circ_list:
        hilbert_dims = 2**circ.width
        
        counts = vector_from_outcomes(data_set[circ].counts, hilbert_dims)
        total_counts = sum(counts)
        observation = (counts + np.ones((hilbert_dims)))/(total_counts + hilbert_dims)
        
        p_model = np.ones((hilbert_dims))
        probs = ref_model.probabilities(circ)
        for key in probs.keys():
            p_model[int(key[0], 2)] += total_counts*probs[key]
        p_model = p_model/(total_counts + hilbert_dims)
        
        pvec = np.hstack((pvec, observation-p_model))
        jmat = np.vstack((jmat, jdict[circ]))
    return np.linalg.pinv(jmat, 1e-4)@pvec
        

# [1] Pick a Model

In [13]:
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XYI
from pygsti.modelpacks import smq1Q_XYZI as _smq1Q_XYZI
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XZ
from pygsti.modelpacks import smq2Q_XYZICNOT as _smq2Q_XYZICNOT
MODEL_PACK = _smq1Q_XYZI
REF_MODEL = MODEL_PACK.target_model('H+S')

# [2] Make Random Circuits and Calculate the Jacobians and Hessians

In [14]:
# make random circuits and design matrices
N_circs = 1000
depth = 15

random_circuits = []
for n in range(N_circs):
    random_circuits.append(random_circuit(random.choice(range(depth)), MODEL_PACK.processor_spec()))

jdict_random = dict()
hdict_random = dict()
for idx, circ in tqdm(enumerate(random_circuits)):
    jdict_random[circ] = matrix_from_jacob(REF_MODEL.sim.dprobs(circ), 2**circ.width)
    hdict_random[circ] = tensor_from_hessian(REF_MODEL.sim.hprobs(circ), 2**circ.width)

1000it [05:26,  3.06it/s]


In [24]:
linear_model = REF_MODEL.copy()
extended_model = REF_MODEL.copy()
regression_model = REF_MODEL.copy()
noise_model = MODEL_PACK.target_model()
noise_model = noise_model.depolarize(max_op_noise=0.01, max_spam_noise=0.001)
noise_model = noise_model.rotate(max_rotate=0.01)    

In [25]:
num_samples = 5000
rb_param, data = make_rb_param(noise_model, MODEL_PACK);
dataset = pig.data.simulate_data(noise_model, random_circuits, num_samples=num_samples)


- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 985242
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 985252
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 985262
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 985272
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 985282


In [26]:

max_iterations = 5
itr_epsilon = 1e-4

num_params = len(REF_MODEL.to_vector())
hilbert_dims = 2

P = 2*np.sqrt(rb_param)*np.eye(num_params)
extended_model.from_vector(np.zeros(num_params))
lkf = LinearKalmanFilter(np.zeros(num_params), P, hilbert_dims)
ekf = ExtendedKalmanFilter(extended_model, P)
ekf2 = ExtendedKalmanFilter(extended_model, P)

In [27]:
for idx, circ in tqdm(enumerate(random_circuits)):
    count_vec = vector_from_outcomes(dataset[circ].counts, hilbert_dims)
    ref_prob = vector_from_outcomes(REF_MODEL.probabilities(circ), hilbert_dims)
    lkf.update_filter(circ, count_vec, ref_prob, jdict_random[circ], 1e-3*np.eye(hilbert_dims))
    ekf.update([circ], dataset, jdict_random, hdict_random, 1e-1*np.eye(hilbert_dims), max_iterations, itr_epsilon)
    ekf2.update([circ], dataset, jdict_random, hdict_random, 1e-1*np.eye(hilbert_dims), max_iterations, itr_epsilon)

1000it [00:17, 58.62it/s]


In [28]:
linear_model.from_vector(lkf.x)
extended_model = ekf.model
extended_model2 = ekf2.model
regression_model.from_vector(regression_fit(random_circuits, dataset, REF_MODEL, jdict_random))

In [29]:
print(regression_model.to_vector())

[-2.91793677e-03 -1.45878205e-03 -3.91342085e-19  1.32363052e-16
  1.30918665e-16 -1.23762142e-18  2.18379216e-03  8.06577349e-04
  1.17061461e-19  1.32128922e-16  1.31559808e-16  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  7.22349268e-03  1.80090875e-04
  5.97923874e-03  3.73130107e-16  3.29772271e-16  3.46303828e-16
  5.71947404e-03  6.39405931e-03 -4.39855571e-04  3.01768648e-16
  3.43812311e-16  2.85237090e-16  3.94330886e-03  4.12119840e-03
  6.24945943e-03  3.65613870e-16  3.82145427e-16  2.98524874e-16]


In [30]:
N_circs = 250
depth = 10
new_circ_list = []
for n in range(N_circs):
    new_circ_list.append(random_circuit(depth, MODEL_PACK.processor_spec()))

In [31]:
%matplotlib
plt.rc('font', size=40)

Using matplotlib backend: Qt5Agg


In [38]:
ref_innovs, linear_innovs = compare_models(noise_model, REF_MODEL, linear_model, new_circ_list, 'Ideal Model Error', 'Linear Kalman Filter Error')

In [39]:
ref_innovs, extended_innovs = compare_models(noise_model, REF_MODEL, extended_model, new_circ_list, 'Ideal Model Error', 'Extended Kalman Filter Error')

In [116]:
ref_innovs, linear_innovs = compare_models(noise_model, REF_MODEL, regression_model, new_circ_list, 'Ideal Model Error', 'Regression Error')